In [355]:
from tools import *
import os
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from tqdm import tqdm  
import random

In [356]:
path="graphes_JSON_Complet/Graphe(5012)_features.json"

node_features_df=extract_node_features_from_json_file(path)
edges_df=extract_mapped_edges_from_json(path)
target_df=extract_optimal_repartition_from_json(path)
node_features_tensor, edge_index_tensor, y_target_tensor = prepare_data_for_GNN(node_features_df, edges_df, target_df)
data = Data(x=node_features_tensor, edge_index=edge_index_tensor, y=y_target_tensor)



print("finished data preparting & loading")

finished data preparting & loading


In [357]:
data

Data(x=[125, 8], edge_index=[2, 266], y=[125])

In [358]:
num_nodes = data.num_nodes
indices = list(range(num_nodes))
random.shuffle(indices)


In [359]:
train_ratio, val_ratio = 0.5, 0
train_size = int(train_ratio * num_nodes)
val_size = int(val_ratio * num_nodes)

train_idx = torch.tensor(indices[:train_size])
val_idx = torch.tensor(indices[train_size:train_size + val_size])
test_idx = torch.tensor(indices[train_size + val_size:])



In [360]:
print(len(train_idx),len(test_idx),len(val_idx))

62 63 0


In [361]:
data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)

In [362]:
data.train_mask[train_idx] = True
# data.val_mask[val_idx] = True
data.test_mask[test_idx] = True

In [363]:
num_features=8
num_classes=int(data.x[0,7])
hidden_channels=64

In [364]:
from torch_geometric.nn.models import GCN, GAT,GraphSAGE

model = GraphSAGE(in_channels=num_features, 
           hidden_channels=64,
           num_layers=4,
           out_channels=num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        pred = out.argmax(dim=1)
        correct = (pred[data.train_mask] == data.y[data.train_mask]).sum()
        acc = int(correct) / int(data.train_mask.sum())

        model.eval()
        with torch.no_grad():
            test_out = model(data.x, data.edge_index)
            test_pred = test_out.argmax(dim=1)
            test_acc = (test_pred[data.test_mask] == data.y[data.test_mask]).float().mean()
        
        print(f'Epoch: {epoch:3d} | Loss: {loss:.4f} | Train Acc: {acc:.4f} | Test Acc: {test_acc:.4f}')

Epoch:   0 | Loss: 8.8864 | Train Acc: 0.0000 | Test Acc: 0.0000
Epoch:  20 | Loss: 3.1909 | Train Acc: 0.1290 | Test Acc: 0.0317
Epoch:  40 | Loss: 2.2768 | Train Acc: 0.2419 | Test Acc: 0.0476
Epoch:  60 | Loss: 1.5912 | Train Acc: 0.5000 | Test Acc: 0.0317
Epoch:  80 | Loss: 0.9610 | Train Acc: 0.7903 | Test Acc: 0.0476
